### Carregando o modelo

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import json
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [2]:
from keras.models import load_model

In [4]:
def loadModel():
    name = input('escreva o nome do modelo a ser carregado:')
    savedModel = load_model(name+'.h5')
    with open('wordIndex.json') as f:
        dicto = json.load(f)
    savedModel.summary()
    return savedModel, dicto

In [5]:
tokenizer = Tokenizer()
model, tokenizer.word_index = loadModel()

escreva o nome do modelo a ser carregado:modelo_truncado500
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          8998800   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 9,116,177
Trainable params: 117,377
Non-trainable params: 8,998,800
_________________________________________________________________


### Notícias analisadas

In [6]:
sentence_t = '''A defensora pública aposentada Cláudia Alvarim Barrozo, 59, gravada agredindo verbalmente entregadores negros, no último sábado (30), em vídeo em que chama um deles de 'macaco', em um condomínio de luxo de Niterói (RJ), tem cinco passagens anteriores pela polícia por injúria racial. Além disso, a reportagem do UOL apurou que as investigações tiveram início em 2014 e que há ainda uma sexta anotação policial antes do caso deste final de semana, mas por lesão corporal e constrangimento.

Claudia Barrozo foi intimada a prestar esclarecimentos na tarde desta quinta-feira (5) na 81ª DP (Itaipu), porém alegou que já tinha "compromissos" e pediu para que a oitiva fosse adiada para a semana que vem.

Hoje pela manhã, Eduardo Pessanha Marques e Jonathas de Souza Mendonça estiveram na delegacia para serem ouvidos novamente. "A sensação que eu tive era de que eu estava em um filme, em uma novela e que aquilo não poderia estar acontecendo", disse Eduardo, que gravou o vídeo flagrando o crime.

De acordo com o delegado Carlos César Santos, atualmente "as pessoas denunciam, não se calam, por isso a incidência [do crime de injúria racial] é maior hoje em dia". Os agentes irão analisar imagens de câmeras de segurança.'''

In [7]:
sentence_f = '''Bolsonaro põe fim ao seu casamento e isolada ex-mulher O casamento de Michelle e Jair Bolsonaro chegou ao fim. O casal passa por uma crise complexa desde novembro de 2018, depois que Jair foi eleito Presidente do Brasil.
 
O site de cobertura e fatos de celebridades e famosos, [site] foi o primeiro site a revelar a separação do casal no dia 13 de outubro de 2020. Na terça-feira o mesmo site revelou que a ex-primeira dama e esposa de Bolsonaro – Michelle Bolsonaro tem sofrido com o fim do casamento e se sentido isolada.

O Custo da vila Olímpica do Japão: 29 milhões de dólares. O custo da vila Olímpica na olimpíada do Brasil: 900 milhões de dólares. e pra variar quem governava o Brasil na época: PT e Lula, quem ganhou muito dinheiro: políticos envolvidos na corrupção, e a globolixo

NO RIO GRANDE DO SUL PRODUTORES DE UVA TOMAM MEDIDAS PARA EVITAR A GEADA NO RIO GRANDE DO SUL PRODUTORES DE UVA TOMAM MEDIDAS PARA EVITAR A GEADA Conforme informações do diário da informação em Bento Gonçalves para não congelar as uvas os produtores estão utilizando o fogo em tonéis, a técnica já é usada há alguns anos no Brasil e em outros países. Além da proteção é mais um espetáculo na Serra Gaúcha.

Lula admite DERROTA em discurso de 1º de Maio. O discurso de Lula que era para ser às 12:00hs foi mudado para às 13:00 e só começou às 15:30. Tudo porque não tinha gente o suficiente. No discurso Lula diz que ainda não é candidato. Ele disse: “só dia 7 eu vou ser pré-candidato”. Será que já arregou?

Bolsonaro põe fim ao seu casamento e isolada ex-mulher O casamento de Michelle e Jair Bolsonaro chegou ao fim. O casal passa por uma crise complexa desde novembro de 2018, depois que Jair foi eleito Presidente do Brasil.
 
O site de cobertura e fatos de celebridades e famosos, [site] foi o primeiro site a revelar a separação do casal no dia 13 de outubro de 2020. Na terça-feira o mesmo site revelou que a ex-primeira dama e esposa de Bolsonaro – Michelle Bolsonaro tem sofrido com o fim do casamento e se sentido isolada.

O Custo da vila Olímpica do Japão: 29 milhões de dólares. O custo da vila Olímpica na olimpíada do Brasil: 900 milhões de dólares. e pra variar quem governava o Brasil na época: PT e Lula, quem ganhou muito dinheiro: políticos envolvidos na corrupção, e a globolixo

NO RIO GRANDE DO SUL PRODUTORES DE UVA TOMAM MEDIDAS PARA EVITAR A GEADA NO RIO GRANDE DO SUL PRODUTORES DE UVA TOMAM MEDIDAS PARA EVITAR A GEADA Conforme informações do diário da informação em Bento Gonçalves para não congelar as uvas os produtores estão utilizando o fogo em tonéis, a técnica já é usada há alguns anos no Brasil e em outros países. Além da proteção é mais um espetáculo na Serra Gaúcha.

Lula admite DERROTA em discurso de 1º de Maio. O discurso de Lula que era para ser às 12:00hs foi mudado para às 13:00 e só começou às 15:30. Tudo porque não tinha gente o suficiente. No discurso Lula diz que ainda não é candidato. Ele disse: “só dia 7 eu vou ser pré-candidato”. Será que já arregou?'''

### Preprocessing

In [8]:
sentence_t = sentence_t.lower()

def remove_punct(raw_string):
    '''Eliminates punctuations and numbers from raw_string.'''
    translator = raw_string.maketrans({key:'' for key in string.punctuation.replace("-", '') + "“”‘’0123456789"})
    
    return raw_string.translate(translator)

sentence_t = remove_punct(sentence_t)

In [9]:
sentence_t

'a defensora pública aposentada cláudia alvarim barrozo  gravada agredindo verbalmente entregadores negros no último sábado  em vídeo em que chama um deles de macaco em um condomínio de luxo de niterói rj tem cinco passagens anteriores pela polícia por injúria racial além disso a reportagem do uol apurou que as investigações tiveram início em  e que há ainda uma sexta anotação policial antes do caso deste final de semana mas por lesão corporal e constrangimento\n\nclaudia barrozo foi intimada a prestar esclarecimentos na tarde desta quinta-feira  na ª dp itaipu porém alegou que já tinha compromissos e pediu para que a oitiva fosse adiada para a semana que vem\n\nhoje pela manhã eduardo pessanha marques e jonathas de souza mendonça estiveram na delegacia para serem ouvidos novamente a sensação que eu tive era de que eu estava em um filme em uma novela e que aquilo não poderia estar acontecendo disse eduardo que gravou o vídeo flagrando o crime\n\nde acordo com o delegado carlos césar sa

In [10]:
sentence_f = sentence_f.lower()

def remove_punct(raw_string):
    '''Eliminates punctuations and numbers from raw_string.'''
    translator = raw_string.maketrans({key:'' for key in string.punctuation.replace("-", '') + "“”‘’0123456789"})  

    return raw_string.translate(translator)

sentence_f = remove_punct(sentence_f)

In [11]:
sentence_f

'bolsonaro põe fim ao seu casamento e isolada ex-mulher o casamento de michelle e jair bolsonaro chegou ao fim o casal passa por uma crise complexa desde novembro de  depois que jair foi eleito presidente do brasil\n \no site de cobertura e fatos de celebridades e famosos site foi o primeiro site a revelar a separação do casal no dia  de outubro de  na terça-feira o mesmo site revelou que a ex-primeira dama e esposa de bolsonaro – michelle bolsonaro tem sofrido com o fim do casamento e se sentido isolada\n\no custo da vila olímpica do japão  milhões de dólares o custo da vila olímpica na olimpíada do brasil  milhões de dólares e pra variar quem governava o brasil na época pt e lula quem ganhou muito dinheiro políticos envolvidos na corrupção e a globolixo\n\nno rio grande do sul produtores de uva tomam medidas para evitar a geada no rio grande do sul produtores de uva tomam medidas para evitar a geada conforme informações do diário da informação em bento gonçalves para não congelar as 

### Classificação Trues

In [12]:
lista_t = []
lista_t.append(sentence_t)
lista_t

['a defensora pública aposentada cláudia alvarim barrozo  gravada agredindo verbalmente entregadores negros no último sábado  em vídeo em que chama um deles de macaco em um condomínio de luxo de niterói rj tem cinco passagens anteriores pela polícia por injúria racial além disso a reportagem do uol apurou que as investigações tiveram início em  e que há ainda uma sexta anotação policial antes do caso deste final de semana mas por lesão corporal e constrangimento\n\nclaudia barrozo foi intimada a prestar esclarecimentos na tarde desta quinta-feira  na ª dp itaipu porém alegou que já tinha compromissos e pediu para que a oitiva fosse adiada para a semana que vem\n\nhoje pela manhã eduardo pessanha marques e jonathas de souza mendonça estiveram na delegacia para serem ouvidos novamente a sensação que eu tive era de que eu estava em um filme em uma novela e que aquilo não poderia estar acontecendo disse eduardo que gravou o vídeo flagrando o crime\n\nde acordo com o delegado carlos césar s

In [15]:
maxlen = 500
sequences_t = tokenizer.texts_to_sequences(lista_t)
padded_t = pad_sequences(sequences_t, maxlen=maxlen)

In [16]:
model.predict(padded_t)

array([[0.1345292]], dtype=float32)

In [17]:
(model.predict(padded_t) >=0.5).astype(int)

array([[0]])

### Classificação Fakes

In [18]:
lista_f = []
lista_f.append(sentence_f)
lista_f

['bolsonaro põe fim ao seu casamento e isolada ex-mulher o casamento de michelle e jair bolsonaro chegou ao fim o casal passa por uma crise complexa desde novembro de  depois que jair foi eleito presidente do brasil\n \no site de cobertura e fatos de celebridades e famosos site foi o primeiro site a revelar a separação do casal no dia  de outubro de  na terça-feira o mesmo site revelou que a ex-primeira dama e esposa de bolsonaro – michelle bolsonaro tem sofrido com o fim do casamento e se sentido isolada\n\no custo da vila olímpica do japão  milhões de dólares o custo da vila olímpica na olimpíada do brasil  milhões de dólares e pra variar quem governava o brasil na época pt e lula quem ganhou muito dinheiro políticos envolvidos na corrupção e a globolixo\n\nno rio grande do sul produtores de uva tomam medidas para evitar a geada no rio grande do sul produtores de uva tomam medidas para evitar a geada conforme informações do diário da informação em bento gonçalves para não congelar as

In [99]:
maxlen = 2000
sequences_f = tokenizer.texts_to_sequences(lista_f)
padded_f = pad_sequences(sequences_f, maxlen=maxlen)

In [100]:
model.predict(padded_f)

array([[0.9751261]], dtype=float32)

In [101]:
(model.predict(padded_f) >=0.5).astype(int)

array([[1]])